<a href="https://colab.research.google.com/github/TheNizzo/SentimentAnalysis/blob/main/NLP_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import random as rdn
import re

In [2]:
!pip install datasets
from datasets import load_dataset
dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')

     |████████████████████████████████| 270 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 32.3 MB/s 
     |████████████████████████████████| 243 kB 69.9 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 125 kB 58.6 MB/s 
     |████████████████████████████████| 109 kB 61.2 MB/s 
     |████████████████████████████████| 546 kB 50.6 MB/s 
     |████████████████████████████████| 160 kB 72.3 MB/s 
     |████████████████████████████████| 294 kB 71.7 MB/s 


Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [3]:
x_train, y_train, x_test, y_test = dataset_train[:]['text'], dataset_train[:]['label'], dataset_test[:]['text'], dataset_test[:]['label']
len(x_train), len(x_test)

(25000, 25000)

In [4]:
0 in y_train

True

#pretreatment

* Take out html

In [5]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [6]:
x_train_pretreated = [remove_tags(el) for el in x_train]
x_test_pretreated = [remove_tags(el) for el in x_test]

# Training

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

In [8]:
import fasttext

In [9]:
def write_list_to_file(filename, list_to_write):
  textfile = open(filename, "w")
  for element in list_to_write:
      textfile.write(element + "\n")
  textfile.close()

In [10]:
def fasttext_conversion(list_y, list_x):
  return ["__label__" + str(y) + " " + x for y, x in zip(list_y, list_x)]

In [11]:
def predict_many(x, y, n, predict_model, verbose=False):
  for i in range(n):
    r = rdn.randint(0,25000)
    print("index = ", r)
    print(predict_model.predict(x[r], k=2), "| Actual value :", y[r])
    if verbose :
      print(x[r])

* No pretreament

In [12]:
train = fasttext_conversion(y_train,x_train)
test = fasttext_conversion(y_test,x_test)

In [13]:
rdn.shuffle(train)
rdn.shuffle(test)

In [14]:
write_list_to_file("train_file.txt", train)
write_list_to_file("test_file.txt", test)

In [15]:
model = fasttext.train_supervised(input="train_file.txt")

In [16]:
predict_many(x_test, y_test, 10, model)

index =  24784
(('__label__0', '__label__1'), array([0.86845464, 0.13156538])) | Actual value : 0
index =  22782
(('__label__0', '__label__1'), array([0.986929  , 0.01309108])) | Actual value : 0
index =  17636
(('__label__0', '__label__1'), array([0.89245832, 0.10756171])) | Actual value : 0
index =  22409
(('__label__0', '__label__1'), array([0.62609756, 0.37392241])) | Actual value : 0
index =  14398
(('__label__0', '__label__1'), array([0.98779941, 0.01222064])) | Actual value : 0
index =  4238
(('__label__1', '__label__0'), array([0.94662845, 0.05339155])) | Actual value : 1
index =  5904
(('__label__0', '__label__1'), array([0.50887734, 0.49114269])) | Actual value : 1
index =  19388
(('__label__0', '__label__1'), array([0.99638706, 0.00363296])) | Actual value : 0
index =  6377
(('__label__1', '__label__0'), array([0.97515798, 0.02486208])) | Actual value : 1
index =  69
(('__label__0', '__label__1'), array([0.80921513, 0.19080482])) | Actual value : 1


In [17]:
model.test("test_file.txt")

(25000, 0.85896, 0.85896)

* With pretreatment

In [18]:
train_pretreated = fasttext_conversion(y_train, x_train_pretreated)
test_pretreated = fasttext_conversion(y_test, x_test_pretreated)

In [19]:
rdn.shuffle(train_pretreated)
rdn.shuffle(test_pretreated)

In [20]:
write_list_to_file("train_file_pretreated.txt", train_pretreated)
write_list_to_file("test_file_pretreated.txt", test_pretreated)

In [21]:
model_pretreated = fasttext.train_supervised(input="train_file_pretreated.txt")

In [22]:
predict_many(x_test_pretreated, y_test, 10, model_pretreated)

index =  8141
(('__label__1', '__label__0'), array([0.83117133, 0.16884865])) | Actual value : 1
index =  4551
(('__label__1', '__label__0'), array([0.87351662, 0.12650341])) | Actual value : 1
index =  12937
(('__label__0', '__label__1'), array([0.76569605, 0.23432393])) | Actual value : 0
index =  2090
(('__label__0', '__label__1'), array([0.80760098, 0.19241902])) | Actual value : 1
index =  6470
(('__label__1', '__label__0'), array([0.86481583, 0.13520412])) | Actual value : 1
index =  19347
(('__label__0', '__label__1'), array([0.98365027, 0.01636971])) | Actual value : 0
index =  14237
(('__label__0', '__label__1'), array([0.9612357 , 0.03878434])) | Actual value : 0
index =  6178
(('__label__1', '__label__0'), array([0.87268734, 0.12733267])) | Actual value : 1
index =  12177
(('__label__1', '__label__0'), array([0.86709899, 0.13292103])) | Actual value : 1
index =  1115
(('__label__1', '__label__0'), array([0.62207931, 0.37794068])) | Actual value : 1


In [23]:
model_pretreated.test("test_file_pretreated.txt")

(25000, 0.85916, 0.85916)

In [24]:
model.words[:50]

['the',
 'a',
 'and',
 'of',
 'to',
 'is',
 'in',
 'I',
 'that',
 'this',
 'it',
 '/><br',
 'was',
 'as',
 'with',
 'for',
 'but',
 'The',
 'on',
 'movie',
 'are',
 'his',
 'film',
 'have',
 'not',
 'be',
 'you',
 '</s>',
 'he',
 'by',
 'at',
 'one',
 'an',
 'from',
 'who',
 'like',
 'all',
 'they',
 'has',
 'so',
 'just',
 'about',
 'or',
 'her',
 'out',
 'some',
 'very',
 'more',
 'This',
 'would']

* Let's add some different parameters

In [30]:
!cat train_file_pretreated.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > train_file_preprocessed.txt
!cat test_file_pretreated.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > test_file_preprocessed.txt

In [31]:
paramed_model = fasttext.train_supervised(input="train_file_preprocessed.txt", lr=0.4, loss="softmax")

In [32]:
paramed_model.test("test_file_preprocessed.txt")

(25000, 0.88472, 0.88472)